## LABORATORIO 0 


MICROESTRUCTURAS Y SISTEMAS DE TRADING

JUAN MANUEL ALVAREZ DEL CASTILLO CASTAÑEDA
IF707620



En el siguiente proyecto se obtiene información histórica de las divisas Dólar-Euro.
A partir de la información se genera un DataFrame con datos relevantes acerca del comportamiento de estos activos para un futuro análisis de éstos.

In [1]:
#%% -- ------------------------------------------------------------- Importar con funciones -- #
import funciones as fn
import visualizaciones as vs
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


# -- --------------------------------------------------------- Descargar precios de OANDA -- #

#%% token de OANDA
OA_Ak = '630eb7b990acb33732bc201e28bf0d80-52218bf769c37309bee9440932b0f008'
OA_In = "EUR_USD"                  # Instrumento
OA_Gn = "D"                       # Granularidad de velas
fini = pd.to_datetime("2019-01-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final

df_pe = fn.f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn,
                             p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=4900)

# -- --------------------------------------------------------------- Graficar OHLC plotly -- #

vs_grafica1 = vs.g_velas(p0_de=df_pe.iloc[0:120, :])
vs_grafica1.show()

# -- ------------------------------------------------------------------- Conteno de velas -- #

#%% multiplicador de precios
pip_mult = 10000

# -- 0A.1: Hora
df_pe['hora'] = [df_pe['TimeStamp'][i].hour for i in range(0, len(df_pe['TimeStamp']))]

# -- 0A.2: Dia de la semana.
df_pe['dia'] = [df_pe['TimeStamp'][i].weekday() for i in range(0, len(df_pe['TimeStamp']))]

# -- 0B: Boxplot de amplitud de velas (close - open).
df_pe['co'] = (df_pe['Close'] - df_pe['Open'])*pip_mult

    
    

TypeError: unsupported operand type(s) for -: 'str' and 'str'

**1. Mes de la Vela**

Se agrega una nueva columna al DataFrame donde se obtiene el mes al que pertenece la vela obteniendo el atributo "month" de la columna TimeStamp

In [2]:
#%%# -- 01: Mes de la vela. 

df_pe['mes'] = [df_pe['TimeStamp'][i].month for i in range(0, len(df_pe['TimeStamp']))]

**2. Sesión de la Vela**

Se agrega una nueva columna al DataFrame donde se obtiene el mes al que pertenece la vela obteniendo el atributo "month" de la columna TimeStamp

In [3]:
#%% -- 02: Sesion de la vela.
df_pe["sesion"] = np.zeros(len(df_pe['High']))

for i in range(0, len(df_pe['High'])):
    
    if  df_pe["hora"][i] > 21 or  df_pe["hora"][i] <8:
        df_pe['sesion'][i] = "asia"
        
    elif  df_pe["hora"][i] == 0:
        df_pe['sesion'][i] = "asia_europa"
        
    elif  df_pe["hora"][i] > 8 and df_pe["hora"][i] <13:
        df_pe['sesion'][i] = "europa"
        
    elif  df_pe["hora"][i] > 12 and df_pe["hora"][i] <17:
        df_pe['sesion'][i] = "europa_america"
        
    else:
        df_pe['sesion'][i] = "america"


C:\Users\juanm\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\juanm\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



**3. Amplitud OC**

Se agrega una nueva columna al DataFrame donde se obtiene la diferencia de precios de Open y Close y se multiplica por 10,000 para representar esta amplitud de la vela en pips.

In [4]:
#%% -- 03: Amplitud OC esperada de vela para cualquier dia de la semana (Dist de Freq).
df_pe['oc'] = [pip_mult*(float(df_pe['Close'][i])-float(df_pe['Open'][i])) for i in range(0, len(df_pe['Close']))]


**4. Amplitud HL**

Al igual que en el punto pasado se obtiene la amplitud pero ahora desde el máximo hasta el mínimo que hubo en ese día

In [5]:
#%% -- 04: Amplitud HL esperada de vela para cualquier dia de la semana (Dist de Freq).
df_pe['hl'] = [pip_mult*(float(df_pe['High'][i])-float(df_pe['Low'][i])) for i in range(0, len(df_pe['High']))]


**5. Evolución de velas consecutivas**

Se pretende saber el número de días consecutivos que se repite una tendencia alcista o bajista.

Para lograr esto se agrega una nueva columna al DataFrame donde se obtiene si la tendencia de ese día fue bajista o alcista y se representa de manera binaria con ceros si fue bajista y unos si fue alcista; esto se logra comparando si el valor de la columna oc es positivo o negativo.

In [6]:
#%% -- 05: Evolucion de velas consecutivas (1: Alcistas, 0: Bajistas).
df_pe["sentido"] = np.where(df_pe["oc"] > 0, 1, 0)

A continuación se compara si el valor de la tendencia observado en la columna "sentido" se repite, si es así se registra una adición al número de días en los que se ha repetido y todo esto se refleja en la nueva columna "sentido_c"

In [7]:
df_pe["sentido_c"] = np.zeros(len(df_pe['High']))

for i in range(1, len(df_pe['High'])):
    
    if  df_pe['sentido'][i] == df_pe['sentido'][i-1]:
        df_pe['sentido_c'][i] = df_pe['sentido_c'][i-1]+1
        
    else:
        df_pe['sentido_c'][i] = 0

C:\Users\juanm\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\juanm\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



**7. Volatilidad Móvil**

Se obtienen tres diferentes ventanas de volatilidad móvil: una de 5 días, otra de 25 y otra más de 50, donde se muestra la volatilidad observada en el periodo correspondiente anterior al día donde observa el dato.

In [8]:
#%% -- 07 Volatilidad Móvil

df_pe["volatilidad5"] = [df_pe.iloc[i-5:i,9].std() for i in range(0, len(df_pe['High']))]
df_pe["volatilidad25"] = [df_pe.iloc[i-25:i,9].std() for i in range(0, len(df_pe['High']))]
df_pe["volatilidad50"] = [df_pe.iloc[i-50:i,9].std() for i in range(0, len(df_pe['High']))]

**8. Gráfica**

Se generó un histograma donde se observa la frecuencia en la que hubo rachas de días consecutivos con la misma tendencia

In [9]:
#%% -- 08: Calculo + Grafica autopropuesta.

fig = px.histogram(df_pe, x= "sentido_c",
                   title='Histograma de repetición de tendencias',
                   labels={'# de días'}, # can specify one label per df column
                           )
fig.update_layout(
    xaxis_title_text='Días Consecutivos Con Misma Tendencia', # xaxis label
    yaxis_title_text='Frecuencia', # yaxis label
    )
    
    
fig.show()